In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model #to load saved models

Using TensorFlow backend.


In [2]:
import os
import pickle
import numpy as np
import json

In [4]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [5]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
model = Sequential()

In [8]:
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [9]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [10]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [11]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [12]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

Using real-time data augmentation.


In [13]:
# Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [16]:
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                                     steps_per_epoch=x_train.shape[0] // batch_size,
                                     epochs=epochs,
                                     validation_data=(x_test, y_test),
                                     workers=4)

Epoch 1/10
1562/1562 [==============================] - 210s - loss: 1.8727 - acc: 0.3105 - val_loss: 1.5910 - val_acc: 0.4305
Epoch 2/10
1562/1562 [==============================] - 206s - loss: 1.5720 - acc: 0.4262 - val_loss: 1.4194 - val_acc: 0.4980
Epoch 3/10
1562/1562 [==============================] - 186s - loss: 1.4391 - acc: 0.4804 - val_loss: 1.3108 - val_acc: 0.5319
Epoch 4/10
1562/1562 [==============================] - 160s - loss: 1.3423 - acc: 0.5212 - val_loss: 1.1598 - val_acc: 0.5928
Epoch 5/10
1562/1562 [==============================] - 159s - loss: 1.2700 - acc: 0.5451 - val_loss: 1.1101 - val_acc: 0.6033
Epoch 6/10
1562/1562 [==============================] - 160s - loss: 1.2063 - acc: 0.5692 - val_loss: 1.0545 - val_acc: 0.6257
Epoch 7/10
1562/1562 [==============================] - 160s - loss: 1.1605 - acc: 0.5860 - val_loss: 1.0318 - val_acc: 0.6356
Epoch 8/10
1562/1562 [==============================] - 160s - loss: 1.1167 - acc: 0.6057 - val_loss: 1.0006 - 

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [19]:
# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

In [18]:
# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Model Accuracy = 0.66
Actual Label = cat vs. Predicted Label = frog
Actual Label = ship vs. Predicted Label = airplane
Actual Label = ship vs. Predicted Label = airplane
Actual Label = airplane vs. Predicted Label = horse
Actual Label = frog vs. Predicted Label = deer
Actual Label = frog vs. Predicted Label = horse
Actual Label = automobile vs. Predicted Label = frog
Actual Label = frog vs. Predicted Label = cat
Actual Label = cat vs. Predicted Label = automobile
Actual Label = automobile vs. Predicted Label = automobile
Actual Label = airplane vs. Predicted Label = bird
Actual Label = truck vs. Predicted Label = frog
Actual Label = dog vs. Predicted Label = ship
Actual Label = horse vs. Predicted Label = dog
Actual Label = truck vs. Predicted Label = horse
Actual Label = ship vs. Predicted Label = airplane
Actual Label = dog vs. Predicted Label = bird
Actual Label = horse vs. Predicted Label = bird
Actual Label = ship vs. Predicted Label = truck
Actual Label = frog vs. Predicted Label

In [ ]:
# Now loading the model from the saved file
model = load_model('keras_cifar10_trained_model.h5')

In [20]:
#Running the model again 


In [24]:
model = Sequential()

In [25]:
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [26]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [27]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [28]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [29]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

Using real-time data augmentation.


In [30]:
# Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [31]:
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                                     steps_per_epoch=x_train.shape[0] // batch_size,
                                     epochs=epochs,
                                     validation_data=(x_test, y_test),
                                     workers=4)

Epoch 1/10
1562/1562 [==============================] - 160s - loss: 2.1407 - acc: 0.2105 - val_loss: 1.9926 - val_acc: 0.2798
Epoch 2/10
1562/1562 [==============================] - 140s - loss: 2.0206 - acc: 0.2621 - val_loss: 1.9186 - val_acc: 0.3072
Epoch 3/10
1562/1562 [==============================] - 166s - loss: 1.9709 - acc: 0.2785 - val_loss: 1.8639 - val_acc: 0.3219
Epoch 4/10
1562/1562 [==============================] - 168s - loss: 1.8973 - acc: 0.3126 - val_loss: 1.7707 - val_acc: 0.3641
Epoch 5/10
1562/1562 [==============================] - 167s - loss: 1.8115 - acc: 0.3445 - val_loss: 1.7233 - val_acc: 0.3929
Epoch 6/10
1562/1562 [==============================] - 168s - loss: 1.7397 - acc: 0.3730 - val_loss: 1.5905 - val_acc: 0.4304
Epoch 7/10
1562/1562 [==============================] - 168s - loss: 1.6871 - acc: 0.3951 - val_loss: 1.6708 - val_acc: 0.4060
Epoch 8/10
1562/1562 [==============================] - 169s - loss: 1.6413 - acc: 0.4079 - val_loss: 1.4919 - 